In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#### Adding Salt and Pepper Noise

In [3]:
import random

In [4]:
def add_noise(image):
    row , col = image.shape
    number_of_pixels = random.randint(300, 5000)
    for i in range(number_of_pixels):
        y_coord=random.randint(0, row - 1)
        x_coord=random.randint(0, col - 1)
        image[y_coord][x_coord] = 255
    number_of_pixels = random.randint(300 , 5000)
    for i in range(number_of_pixels):
        y_coord=random.randint(0, row - 1)
        x_coord=random.randint(0, col - 1)
        image[y_coord][x_coord] = 0
        
    return image

In [5]:
import cv2
image = cv2.imread(r"G://OVGU Study//Sem2//Introduction to Computer Vision//Assignments//assign1//image.png", cv2.IMREAD_GRAYSCALE)
cv2.imshow("image",image)
cv2.waitKey(0)

-1

In [6]:
sap_car = add_noise(image)
cv2.imshow("sap_car",sap_car)
cv2.waitKey(0)

-1

In [7]:
#### Applying Gaussian Filter to Salt and Pepper Noise Image

In [8]:
### Implementation with OpenCV package - Gaussian Filter

In [9]:
gaussian_cv_filter = cv2.GaussianBlur(sap_car,(5,5),3)

In [10]:
cv2.imshow("gaussian_cv_filter", gaussian_cv_filter)
cv2.waitKey(0)

-1

In [22]:
### Implementation without OpenCV package - Gaussian Filter - not working

In [11]:
def convolution(oldimage, kernel):
    #image = Image.fromarray(image, 'RGB')
    image_h = oldimage.shape[0]
    #image_w = oldimage.shape[1]
    
    
    kernel_h = kernel.shape[0]
    kernel_w = kernel.shape[1]
    
    if(len(oldimage.shape) == 3):
        image_pad = np.pad(oldimage, pad_width=((kernel_h // 2, kernel_h // 2),(kernel_w // 2, kernel_w // 2),(0,0)), mode='constant', constant_values=0).astype(np.float32)
    elif(len(oldimage.shape) == 2):
        image_pad = np.pad(oldimage, pad_width=((kernel_h // 2, kernel_h // 2),(kernel_w // 2, kernel_w // 2)), mode='constant', constant_values=0).astype(np.float32)
    
    
    h = kernel_h // 2
    w = kernel_w // 2
    
    image_conv = np.zeros(image_pad.shape)
    
    for i in range(h, image_pad.shape[0]-h):
        for j in range(w, image_pad.shape[1]-w):
            #sum = 0
            x = image_pad[i-h:i-h+kernel_h, j-w:j-w+kernel_w]
            x = x.flatten()*kernel.flatten()
            image_conv[i][j] = x.sum()
    h_end = -h
    w_end = -w
    
    if(h == 0):
        return image_conv[h:,w:w_end]
    if(w == 0):
        return image_conv[h:h_end,w:]
    return image_conv[h:h_end,w:w_end]

In [12]:
def GaussianBlurImage(image, sigma):
    #image = imread(image)
    #image = Image.open(image)
    image = np.asarray(image)
    
    filter_size = 2 * int(4 * sigma + 0.5) + 1
    gaussian_filter = np.zeros((filter_size, filter_size), np.float32)
    m = filter_size//2
    n = filter_size//2
    
    for x in range(-m, m+1):
        for y in range(-n, n+1):
            x1 = 2*np.pi*(sigma**2)
            x2 = np.exp(-(x**2 + y**2)/(2* sigma**2))
            gaussian_filter[x+m, y+n] = (1/x1)*x2
    
    im_filtered = np.zeros_like(image, dtype=np.float32)
    for c in range(2):
        im_filtered[ :, c] = convolution(image[ :, c], gaussian_filter)
    return (im_filtered.astype(np.uint8))

In [14]:
gaus_without_cv = GaussianBlurImage(sap_car, 3)

In [15]:
#### Applying Bilateral Filter to Salt and Pepper Noise Image

In [16]:
### Implementation with OpenCV package - Bilateral Filter

In [17]:
bilateral_cv_filter = cv2.bilateralFilter(sap_car,15,80,80)

In [18]:
cv2.imshow("bilateral_cv_filter", bilateral_cv_filter)
cv2.waitKey(0)

-1

In [21]:
### Implementation without OpenCV package - Bilateral Filter - not working

In [20]:
import math
import sys
 
 
def vec_gaussian(img: np.ndarray, variance: float) -> np.ndarray:
    # For applying gaussian function for each element in matrix.
    sigma = math.sqrt(variance)
    cons = 1 / (sigma * math.sqrt(2 * math.pi))
    return cons * np.exp(-((img / sigma) ** 2) * 0.5)
 
 
def get_slice(img: np.ndarray, x: int, y: int, kernel_size: int) -> np.ndarray:
    half = kernel_size // 2
    return img[x - half : x + half + 1, y - half : y + half + 1]
 
 
def get_gauss_kernel(kernel_size: int, spatial_variance: float) -> np.ndarray:
    # Creates a gaussian kernel of given dimension.
    arr = np.zeros((kernel_size, kernel_size))
    for i in range(0, kernel_size):
        for j in range(0, kernel_size):
            arr[i, j] = math.sqrt(
                abs(i - kernel_size // 2) ** 2 + abs(j - kernel_size // 2) ** 2
            )
    return vec_gaussian(arr, spatial_variance)
 
 
def bilateral_filter(
    img: np.ndarray,
    spatial_variance: float,
    intensity_variance: float,
    kernel_size: int,
) -> np.ndarray:
    img2 = np.zeros(img.shape)
    gaussKer = get_gauss_kernel(kernel_size, spatial_variance)
    sizeX, sizeY = img.shape
    for i in range(kernel_size // 2, sizeX - kernel_size // 2):
        for j in range(kernel_size // 2, sizeY - kernel_size // 2):
 
            imgS = get_slice(img, i, j, kernel_size)
            imgI = imgS - imgS[kernel_size // 2, kernel_size // 2]
            imgIG = vec_gaussian(imgI, intensity_variance)
            weights = np.multiply(gaussKer, imgIG)
            vals = np.multiply(imgS, weights)
            val = np.sum(vals) / np.sum(weights)
            img2[i, j] = val
    return img2
 
 
def parse_args(args: list) -> tuple:
    filename = args[1] if args[1:] else "../image_data/lena.jpg"
    spatial_variance = float(args[2]) if args[2:] else 1.0
    intensity_variance = float(args[3]) if args[3:] else 1.0
    if args[4:]:
        kernel_size = int(args[4])
        kernel_size = kernel_size + abs(kernel_size % 2 - 1)
    else:
        kernel_size = 5
    return filename, spatial_variance, intensity_variance, kernel_size
 
 
if __name__ == "__main__":
    #filename, 
    spatial_variance, intensity_variance, kernel_size = parse_args(sys.argv)
#     img = cv2.imread(filename, 0)
#     cv2.imshow("input image", img)
 
    out = sap_car / 255
    out = out.astype("float32")
    out = bilateral_filter(out, spatial_variance, intensity_variance, kernel_size)
    out = out * 255
    out = np.uint8(out)
    cv2.imshow("output image", out)
    cv2.waitKey(0)
    cv2.destroyAllWindows()